In [1]:
from dotenv import load_dotenv
import os
from pathlib import Path
from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
from sqlalchemy import create_engine
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [2]:
local_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(local_dir)
config_file_path = os.path.join(parent_dir,'API.env')
load_dotenv(Path(config_file_path))

True

In [3]:
local_dir = os.path.dirname(os.path.abspath('__file__'))
print(os.path.dirname(local_dir))

/Users/ml1000278614/Documents/PythonProjects/stock_paper_trader


In [5]:
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")
base_url = os.getenv("BASE_URL")

api_key

'PKZ8VHIL01SZ5OACM0VR'

In [13]:
from google.cloud import storage

In [53]:
data = {'stock':['btc-usd'],'action':['sell'],'price':[2450.19],'date':['01-05-2024']}
df = pd.DataFrame(data)
df

,stock,action,price,date
0,btc-usd,sell,2450.19,01-05-2024


In [54]:
client = storage.Client()

bucket_name = 'trade-result-bucket'
blob_name = 'trade-results.db'

table_name = 'results'
url = 'sqlite:///temp_db_file.db'

# Connect to google cloud bucket
bucket = client.get_bucket('trade-result-bucket')
blob = bucket.blob(blob_name)

# Check if blob exists
if blob.exists():
    blob.download_to_filename("temp_db_file.db")

engine = create_engine(url)

df.to_sql(table_name,con=engine,if_exists='append',index=False)

# Upload updated db file
blob.upload_from_filename("temp_db_file.db")

# Remove File
os.remove("temp_db_file.db")



In [14]:
client = storage.Client()

bucket_name = 'trade-result-bucket'
blob_name = 'trade-results.db'

table_name = 'results'
url = 'sqlite:///temp_db_file.db'

# Connect to google cloud bucket
bucket = client.get_bucket('trade-result-bucket')
blob = bucket.blob(blob_name)

blob.download_to_filename("temp_db_file.db")

engine = create_engine(url)
query = f"SELECT * FROM {table_name}"

pd.read_sql(query,engine)

,stock,action,price,date
0,BTC/USD,None,62489.955,2024-05-08 05:00:00.000000
1,BTC/USD,None,62489.955,2024-05-08 05:00:00.000000


In [6]:
client = CryptoHistoricalDataClient()
stock_client = StockHistoricalDataClient(api_key,api_secret)

In [7]:
request_params = CryptoBarsRequest(
    symbol_or_symbols=["BTC/USD"],
    timeframe=TimeFrame.Day,
    start="2020-01-01"
)

btc_bars = client.get_crypto_bars(request_params=request_params)
df = btc_bars.df

In [66]:
df.reset_index(inplace=True)

In [67]:
df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,BTC/USD,2021-01-01 06:00:00+00:00,29255.7100,29682.29000,28707.5600,29676.790,848.874030,29639.0,29316.444625
1,BTC/USD,2021-01-02 06:00:00+00:00,29678.3400,34200.00000,29555.9900,33769.520,2144.592516,60152.0,31941.412694
2,BTC/USD,2021-01-03 06:00:00+00:00,33769.5200,34812.93000,32300.6100,32908.020,1838.695433,58725.0,33505.269474
3,BTC/USD,2021-01-04 06:00:00+00:00,32907.3600,33496.03000,27900.0000,30441.570,2711.189503,69226.0,31267.965121
4,BTC/USD,2021-01-05 06:00:00+00:00,30461.8400,35851.42000,29927.5500,35063.000,1756.751333,61880.0,33151.367357
...,...,...,...,...,...,...,...,...,...
1219,BTC/USD,2024-05-04 05:00:00+00:00,63107.5095,64545.42500,62928.4145,63351.590,1.532918,89.0,63693.674070
1220,BTC/USD,2024-05-05 05:00:00+00:00,63389.6760,64643.33300,63164.3000,64163.150,0.459625,51.0,64238.200244
1221,BTC/USD,2024-05-06 05:00:00+00:00,64133.6520,65455.64194,62757.9390,63453.800,2.077109,256.0,64435.103616
1222,BTC/USD,2024-05-07 05:00:00+00:00,63422.5900,64356.47800,62171.5750,62510.200,0.656296,99.0,63067.503531


In [6]:
trading_client = TradingClient(api_key,api_secret,paper=True)

In [9]:
market_order_data = MarketOrderRequest(
    symbol='BTCUSD',
    qty=0.23,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.GTC
)
market_order = trading_client.submit_order(
    order_data=market_order_data
)

In [11]:
pos = trading_client.get_open_position('BTCUSD')

market_order_data = MarketOrderRequest(
    symbol='BTCUSD',
    qty=pos.qty,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.GTC
)

market_order = trading_client.submit_order(
    order_data=market_order_data
)